In [1]:
from spacetrack import SpaceTrackClient
import pandas as pd

from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder.appName('SattelitesProject').getOrCreate()

st = SpaceTrackClient('hassansari4a@gmail.com', '9817377317NCELL')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 13:27:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [148]:
csvfile = st.satcat(format='csv')

In [149]:
with open('my_file.csv', 'w') as my_file:
    my_file.write(csvfile)

In [150]:
df99 = pd.read_csv('my_file.csv')
df99.to_csv('satcatdata.csv', index=False)

/var/folders/27/yj55qwjs7md5w890736t7nc40000gn/T/ipykernel_19454/1457401881.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df99 = pd.read_csv('my_file.csv')


In [3]:
dfspark = spark.read.format("csv") \
                   .option("delimiter", ",") \
                   .option("header", "true") \
                   .option("inferSchema", "true") \
                   .load("satcatdata.csv")

In [152]:
dfspark.

+---------+------------+-----------+--------------------+-------+-------------------+-----+-------------------+--------+-----------+--------+-------+-------+-----------+--------+--------+----+-----------+----------+------------+-------+--------------------+---------+-------------+
|  INTLDES|NORAD_CAT_ID|OBJECT_TYPE|             SATNAME|COUNTRY|             LAUNCH| SITE|              DECAY|  PERIOD|INCLINATION|  APOGEE|PERIGEE|COMMENT|COMMENTCODE|RCSVALUE|RCS_SIZE|FILE|LAUNCH_YEAR|LAUNCH_NUM|LAUNCH_PIECE|CURRENT|         OBJECT_NAME|OBJECT_ID|OBJECT_NUMBER|
+---------+------------+-----------+--------------------+-------+-------------------+-----+-------------------+--------+-----------+--------+-------+-------+-----------+--------+--------+----+-----------+----------+------------+-------+--------------------+---------+-------------+
|1957-001A|           1|ROCKET BODY|            SL-1 R/B|    CIS|1957-10-04 00:00:00|TTMTR|1957-12-01 00:00:00|   96.19|       65.1|   938.0|  214.0|   nu

In [155]:
dfspark.head(3)

[Row(INTLDES='1957-001A', NORAD_CAT_ID=1, OBJECT_TYPE='ROCKET BODY', SATNAME='SL-1 R/B', COUNTRY='CIS', LAUNCH=datetime.datetime(1957, 10, 4, 0, 0), SITE='TTMTR', DECAY=datetime.datetime(1957, 12, 1, 0, 0), PERIOD=96.19, INCLINATION=65.1, APOGEE=938.0, PERIGEE=214.0, COMMENT=None, COMMENTCODE=4.0, RCSVALUE=0, RCS_SIZE='LARGE', FILE=1, LAUNCH_YEAR=1957, LAUNCH_NUM=1, LAUNCH_PIECE='A', CURRENT='Y', OBJECT_NAME='SL-1 R/B', OBJECT_ID='1957-001A', OBJECT_NUMBER=1),
 Row(INTLDES='1958-001A', NORAD_CAT_ID=4, OBJECT_TYPE='PAYLOAD', SATNAME='EXPLORER 1', COUNTRY='US', LAUNCH=datetime.datetime(1958, 2, 1, 0, 0), SITE='AFETR', DECAY=datetime.datetime(1970, 3, 31, 0, 0), PERIOD=88.48, INCLINATION=33.15, APOGEE=215.0, PERIGEE=183.0, COMMENT=None, COMMENTCODE=None, RCSVALUE=0, RCS_SIZE=None, FILE=1, LAUNCH_YEAR=1958, LAUNCH_NUM=1, LAUNCH_PIECE='A', CURRENT='Y', OBJECT_NAME='EXPLORER 1', OBJECT_ID='1958-001A', OBJECT_NUMBER=4),
 Row(INTLDES='1958-003A', NORAD_CAT_ID=6, OBJECT_TYPE='PAYLOAD', SATNAME=

In [4]:
dfspark.describe()

DataFrame[summary: string, INTLDES: string, NORAD_CAT_ID: string, OBJECT_TYPE: string, SATNAME: string, COUNTRY: string, SITE: string, PERIOD: string, INCLINATION: string, APOGEE: string, PERIGEE: string, COMMENT: string, COMMENTCODE: string, RCSVALUE: string, RCS_SIZE: string, FILE: string, LAUNCH_YEAR: string, LAUNCH_NUM: string, LAUNCH_PIECE: string, CURRENT: string, OBJECT_NAME: string, OBJECT_ID: string, OBJECT_NUMBER: string]

In [13]:
#Drop columns 'COMMENT', 'COMMENTCODE', 'RCSVALUE', 'RCS_SIZE'

dfspark = dfspark.drop('COMMENT', 'COMMENTCODE', 'RCSVALUE', 'RCS_SIZE')

In [14]:
dfspark.show()

+---------+------------+-----------+--------------------+-------+-------------------+-----+-------------------+--------+-----------+--------+-------+----+-----------+----------+------------+-------+--------------------+---------+-------------+
|  INTLDES|NORAD_CAT_ID|OBJECT_TYPE|             SATNAME|COUNTRY|             LAUNCH| SITE|              DECAY|  PERIOD|INCLINATION|  APOGEE|PERIGEE|FILE|LAUNCH_YEAR|LAUNCH_NUM|LAUNCH_PIECE|CURRENT|         OBJECT_NAME|OBJECT_ID|OBJECT_NUMBER|
+---------+------------+-----------+--------------------+-------+-------------------+-----+-------------------+--------+-----------+--------+-------+----+-----------+----------+------------+-------+--------------------+---------+-------------+
|1957-001A|           1|ROCKET BODY|            SL-1 R/B|    CIS|1957-10-04 00:00:00|TTMTR|1957-12-01 00:00:00|   96.19|       65.1|   938.0|  214.0|   1|       1957|         1|           A|      Y|            SL-1 R/B|1957-001A|            1|
|1958-001A|           4|

In [25]:
dfspark.describe()

DataFrame[summary: string, INTLDES: string, NORAD_CAT_ID: string, OBJECT_TYPE: string, SATNAME: string, COUNTRY: string, SITE: string, PERIOD: string, INCLINATION: string, APOGEE: string, PERIGEE: string, FILE: string, LAUNCH_YEAR: string, LAUNCH_NUM: string, LAUNCH_PIECE: string, CURRENT: string, OBJECT_NAME: string, OBJECT_ID: string, OBJECT_NUMBER: string]

In [42]:
dfspark.groupby('OBJECT_TYPE').count().show()

+-----------+-----+
|OBJECT_TYPE|count|
+-----------+-----+
|     DEBRIS|34360|
|    UNKNOWN|  425|
|    PAYLOAD|14822|
|ROCKET BODY| 6442|
+-----------+-----+



In [52]:
dfspark.write.parquet('satcatdata.parquet')

TypeError: PandasConversionMixin.toPandas() got an unexpected keyword argument 'index'

In [53]:
df

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,COUNTRY,LAUNCH,SITE,DECAY,PERIOD,INCLINATION,APOGEE,PERIGEE,FILE,LAUNCH_YEAR,LAUNCH_NUM,LAUNCH_PIECE,CURRENT,OBJECT_NAME,OBJECT_ID,OBJECT_NUMBER
0,1957-001A,1,ROCKET BODY,SL-1 R/B,CIS,1957-10-04,TTMTR,1957-12-01,96.19,65.10,938.0,214.0,1,1957,1,A,Y,SL-1 R/B,1957-001A,1
1,1958-001A,4,PAYLOAD,EXPLORER 1,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215.0,183.0,1,1958,1,A,Y,EXPLORER 1,1958-001A,4
2,1958-003A,6,PAYLOAD,EXPLORER 3,US,1958-03-26,AFETR,1958-06-28,103.60,33.50,1739.0,117.0,1,1958,3,A,Y,EXPLORER 3,1958-003A,6
3,1958-004A,7,ROCKET BODY,SL-1 R/B,CIS,1958-05-15,TTMTR,1958-12-03,102.74,65.14,1571.0,206.0,1,1958,4,A,Y,SL-1 R/B,1958-004A,7
4,1958-004B,8,PAYLOAD,SPUTNIK 3,CIS,1958-05-15,TTMTR,1960-04-06,88.43,65.06,255.0,139.0,1,1958,4,B,Y,SPUTNIK 3,1958-004B,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56044,2023-050F,56167,UNKNOWN,OBJECT F,US,2023-04-02,AFWTR,NaT,103.99,81.01,952.0,942.0,8373,2023,50,F,Y,OBJECT F,2023-050F,56167
56045,2023-050G,56168,UNKNOWN,OBJECT G,US,2023-04-02,AFWTR,NaT,103.98,81.00,951.0,942.0,8373,2023,50,G,Y,OBJECT G,2023-050G,56168
56046,2023-050H,56169,UNKNOWN,OBJECT H,US,2023-04-02,AFWTR,NaT,103.98,81.00,951.0,942.0,8373,2023,50,H,Y,OBJECT H,2023-050H,56169
56047,2023-050J,56170,UNKNOWN,OBJECT J,US,2023-04-02,AFWTR,NaT,103.94,81.00,950.0,939.0,8373,2023,50,J,Y,OBJECT J,2023-050J,56170


In [54]:
dfspark.groupby('CURRENT').count().show()

+-------+-----+
|CURRENT|count|
+-------+-----+
|      Y|56049|
+-------+-----+



In [57]:
dfspark.coalesce(1).write.parquet("../data/satcatdata")


AnalysisException: path file:/Users/hassanmuhammed/Desktop/Home/Repositories/SattelitesProject/data/satcatdata already exists.